In [6]:
import sys
sys.path.append('/home/ubuntu/jjlee/jumpstart-rl/')
from experiments.gridworld import ContinuousGridworld
from gymnasium.core import Wrapper
from stable_baselines3 import SAC
from gym.core import Env
from src.jsrl.jsrl import get_jsrl_algorithm

env = ContinuousGridworld(seed=1)

def extract_path(policy):
    x_path = []
    y_path = []
    state = env.reset()
    while True:
        action = policy.predict([state])
        state, _, done, _, _ = env.step(action)
        x_path.append(env.agent_position[0])
        y_path.append(env.agent_position[1])
        if done:
            break
    return x_path, y_path
x, y = extract_path(env.bad_oracle)
env.render_path(x, y)


IndexError: invalid index to scalar variable.

In [ ]:
def extract_path(policy):
    x_path = []
    y_path = []
    state = env.reset()
    while True:
        action = policy.predict()
        state, _, done, _ = env.step(action)
        x_path.append(env.agent_position[0])
        y_path.append(env.agent_position[1])
        if done:
            break
    return x_path, y_path
x, y = extract_path()
env.render_path(x, y)